# AIM - To create a ML model to identify FM stations

Python version used -> python 3

# Step - 1
## Run RTL_power command 

Install GQRX software then connect RTL_SDR dongle and open terminal.

Note : Following command only works for Linux and Mac OS.

COMMAND -> rtl_power -f min:max:bin -g gain -i interval -e runtime filename.ext
where
min is initial frequency
max is terminal frequency
bin is frequency interval
interval in seconds

COMMAND I USED - 

rtl_power -f 87M:108M:1k -g 20 -i 10 -e 5m logfile.csv

All the data is stored in a csv file logfile.csv.

# Step - 2
## Data cleaning 

We will now convert obtained csv into a desireable pandas dataframe

In [1]:
import numpy as np
import pandas as pd
dfs = pd.read_csv("logfile.csv", header=None)

In [3]:
def conversion_function(df):    
    x=[] # Stores all the frequencies
    y=[] # Stores corresponding power value
    z=[] # Stores is_FM, if 1 then yes, if 0 then no

    # following array contains all BAY AREA FM STATIONS
    arr = [87.9, 88.1, 88.5, 89.1, 89.3, 89.5, 89.7, 89.9, 90.1, 90.3, 90.5, 90.7, 91.1, 91.5, 91.7, 92.1, 92.3, 92.7, 93.3, 94.1, 94.5, 94.9, 95.3, 95.7, 96.1, 96.5, 97.3, 98.1, 98.5, 98.9, 99.7, 100.3, 101.3, 101.7, 102.1, 102.9, 103.3, 103.7, 104.5, 104.9, 105.3, 105.7, 106.1, 106.5, 106.9, 107.7]

    #following code makes 3 lists, x-> freq,y-> power, z-> Is_FM?
    for j in range(0,len(df)):
        for i in range(6,4103):
            y.append(float(df[i][j]))
            r = (df[3][j]-df[2][j])/4096
            temp = df[3][j]+(r*(i-6))
            x.append(temp)
            check = round(temp/100000)
            check = int(check)
            check = float(check/10)
            n=0
            if(check in arr):
                n=1
            z.append(int(n))
    df = pd.DataFrame({"Frequency":x,"Power":y,"Is_FM":z})
    return df

In [4]:
%%time
dfs = conversion_function(dfs)

Wall time: 1min 52s


# Step 3 - 
## Train all the classification models to find the best one

Note -  For training, frequency range is from 87 MHz to 108 MHz

In [6]:
%%time
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

Wall time: 702 ms


D:\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
X = dfs.drop('Is_FM', axis=1)
y = dfs['Is_FM']

In [8]:
%%time

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X, y)

y_predict = knn.predict(X)
print("Using KNN -\nTraining accuracy :",accuracy_score(y,y_predict)*100,"%")

Using Logistic regression -
Training accuracy : 80.9555772516 %
Using KNN -
Training accuracy : 100.0 %
Using Perceptron -
Training accuracy : 80.9555772516 %
Using XGBoost -
Training accuracy : 100.0 %
Wall time: 5min 57s


In [ ]:
from sklearn.externals import joblib
joblib.dump(knn, 'findingFMstations_trainedmodel.pkl') 

# Step - 4
## Testing with test data in frequency range 87MHz to 108MHz

Command used to generate file -

rtl_power -f 87M:108M:1k -g 20 -i 10 -e 5m lgfile.csv

Generating list of fm stations recognised -

In [2]:
from sklearn.externals import joblib
knn = joblib.load('findingFMstations_trainedmodel.pkl')
dfs = pd.read_csv("logfile2.csv", header=None)

In [3]:
def conv_func(df):
    x=[] # Stores all the frequencies
    y=[] # Stores corresponding power value
    
    for j in range(0,len(df)):
        for i in range(6,4103):
            y.append(float(df[i][j]))
            r = (df[3][j]-df[2][j])/4096
            temp = df[3][j]+(r*(i-6))
            x.append(temp)
    df = pd.DataFrame({"Frequency":x,"Power":y})
    return df

In [4]:
%%time
dfs = conv_func(dfs)

Wall time: 30.5 s


In [6]:
%%time
y_predict3 = knn.predict(dfs)

Wall time: 1.47 s


In [70]:
y_predict33 = knn.predict(dfs1)

In [71]:
%%time
l3=[]
for i in range(0,len(y_predict3)):
    num = round(dfs["Frequency"][i]/100000)
    num = int(num)
    num = float(num/10)
    if(y_predict3[i]==1):
        if(not num in l3):
            l3.append(num)

Wall time: 7.62 s


In [76]:
#Bay Area FM Radio Station
radio = [87.9, 88.1, 88.5, 89.1, 89.3, 89.5, 89.7, 89.9, 90.1, 90.3, 90.5, 90.7, 91.1, 91.5, 91.7, 92.1, 92.3, 92.7, 93.3, 94.1, 94.5, 94.9, 95.3, 95.7, 96.1, 96.5, 97.3, 98.1, 98.5, 98.9, 99.7, 100.3, 101.3, 101.7, 102.1, 102.9, 103.3, 103.7, 104.5, 104.9, 105.3, 105.7, 106.1, 106.5, 106.9, 107.7]

In [77]:
ctr= 0
for i in range (0,len(l1)):
    if(l1[i] in radio):
        ctr=ctr+1
print("For ")
print("XGBOOST found", len(l1),"radio stations out of which actual Bay Area FM Radio Station are",ctr,"\nAnd",ctr,"stations were found out of",len(radio),"Bay Area FM Radio Station\nXGBOOST accuracy -",ctr/len(l1)*100,"%")
ctr=0
for i in range (0,len(l3)):
    if(l3[i] in radio):
        ctr=ctr+1
print("KNN model found", len(l3),"radio stations out of which actual Bay Area FM Radio Station are",ctr,"\nAnd",ctr,"stations were found out of",len(radio),"Bay Area FM Radio Station\nKNN model accuracy -",ctr/len(l3)*100,"%")

XGBOOST found 40 radio stations out of which actual Bay Area FM Radio Station are 40 
And 40 stations were found out of 46 Bay Area FM Radio Station
XGBOOST accuracy - 100.0 %
KNN model found 40 radio stations out of which actual Bay Area FM Radio Station are 40 
And 40 stations were found out of 46 Bay Area FM Radio Station
KNN model accuracy - 100.0 %
